In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

import time

num_images = 10
pause_time = 2

filename = os.path.relpath('../../CameraCalibration/Calibration3D');


In [2]:
sourceL = cv2.VideoCapture(0)
sourceR = cv2.VideoCapture(1)

[ WARN:0@1.443] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@3.085] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [6]:
time.sleep(pause_time)

for idx in range(0,num_images):
    filename_localL = filename +'L'+ str(idx) + '.jpg'
    filename_localR = filename +'R'+ str(idx) + '.jpg'
    has_frame, cur_frameL = sourceL.read()
    has_frame, cur_frameR = sourceR.read()
    
    cur_frameL = cv2.cvtColor(cur_frameL,cv2.COLOR_BGR2GRAY)
    
    cur_frameR = cv2.cvtColor(cur_frameR,cv2.COLOR_BGR2GRAY)

    print(idx)

    cv2.imwrite(filename_localL,cur_frameL)
    cv2.imwrite(filename_localR,cur_frameR)
    time.sleep(pause_time)

0
1
2
3
4
5
6
7
8
9


In [8]:
import glob

img_filenamesL = glob.glob(os.path.relpath(filename + "L*.jpg"))
img_filenamesL=sorted(img_filenamesL)

img_filenamesR = glob.glob(os.path.relpath(filename + "R*.jpg"))
img_filenamesR=sorted(img_filenamesR)



img_listL = [cv2.imread(file) for file in img_filenamesL]
img_listL=[cv2.cvtColor(img_listL[idx],cv2.COLOR_BGR2GRAY) for idx in range(0,len(img_listL))]

img_listR = [cv2.imread(file) for file in img_filenamesR]
img_listR=[cv2.cvtColor(img_listR[idx],cv2.COLOR_BGR2GRAY) for idx in range(0,len(img_listR))]

In [10]:
chessboard_size = (8, 6)

square_size = 25  # mm


criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points,
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints1 = []  # 2d points in image plane for camera 1
imgpoints2 = []  # 2d points in image plane for camera 2

# Get image file names

for idx in range(0,len(img_listL)):
    img1 = img_listL[idx]
    img2 = img_listR[idx]
    
    # Find the chess board corners
    ret1, corners1 = cv2.findChessboardCorners(img1, chessboard_size, None)
    ret2, corners2 = cv2.findChessboardCorners(img2, chessboard_size, None)

    # If found, add object points, image points (after refining them)
    if ret1 and ret2:
        objpoints.append(objp)
        corners1 = cv2.cornerSubPix(img1, corners1, (11, 11), (-1, -1), criteria)
        imgpoints1.append(corners1)
        corners2 = cv2.cornerSubPix(img2, corners2, (11, 11), (-1, -1), criteria)
        imgpoints2.append(corners2)

    # Calibrate cameras
    ret1, mtx1, dist1, rvecs1, tvecs1 = cv2.calibrateCamera(objpoints, imgpoints1, img1.shape[::-1], None, None)
    ret2, mtx2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(objpoints, imgpoints2, img2.shape[::-1], None, None)

    # Stereo calibration
    retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(
        objpoints, imgpoints1, imgpoints2, mtx1, dist1, mtx2, dist2, img1.shape[::-1], criteria )
